# 0. Preamble

In [1]:
import os
import logging
import pandas as pd
import numpy as np

import thomas.core

logging_setup = False

# Functions below based on 
# https://stackoverflow.com/questions/12544056/how-do-i-get-the-current-ipython-jupyter-notebook-name
def get_kernel_id():
    import ipykernel
    
    connection_file = os.path.basename(ipykernel.get_connection_file())
    return connection_file.split('-', 1)[1].split('.')[0]

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    from notebook import notebookapp
    import urllib
    import json

    kernel_id = get_kernel_id()
    
    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
        
    return None

In [2]:
print(f'connect to this kernel with:\n  jupyter console --existing {get_kernel_id()}')

connect to this kernel with:
  jupyter console --existing 5adde929-5608-440f-b3ef-e38428812bf5


In [3]:
if logging_setup == False:
    log = logging.getLogger()

    # handler = OutputWidgetHandler()
    logfile = os.path.splitext(notebook_path())[0] + '.log'
    
    # Delete the logfile to start fresh.
    # Use tail -F to follow the file regardless :-)
    try:
       os.remove(logfile)
    except:
       pass

    with open(logfile, 'w') as fp:
        fp.write('\n')
    
    handler = logging.FileHandler(logfile)
    handler.setFormatter(
        logging.Formatter(
            '%(asctime)s  - %(name)-14s - [%(levelname)s] %(message)s',
            "%Y-%m-%d %H:%M:%S")
    )
    log.addHandler(handler)
    log.setLevel(logging.INFO)
    logging_setup = True
    
    # Clean up
    del fp, logfile


log.info('-' * 60)
log.info(f'Starting notebook "{os.path.split(notebook_path())[-1] }"')
log.info(f'  thomas.core version: {thomas.core.__version__}')
log.info(f'  Pandas version: {pd.__version__}')
log.info(f'  Current working directory: {os.getcwd()}')
log.info('-' * 60)

In [4]:
# handler.clear_logs()
# for x in range(200):
#     log.debug(f'{x:02}: yeaj')